<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 6: Create a Differentially Private Query
Use Laplacian noise to create a query function which sums over the database and adds the right amount of noise to the query output (Global Differential Privacy) such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.
### GLOSSARY
**Randomized machanizm** is a function with random noise added to its inouts, outputs, and/or inner workings.

**Privacy budget** will tell you how much epsilon/delta leakage we allow for our analysis.

**Types of noise**
- Gaussian
- Laplacian (workls better)   __***b = sensitivity(query)/epsilon***__ (delta == 0 therefore it's omitted)

**How much noise should we add**?
- Type of noise (Gaussian / Laplacian) (type of noise we are adding)
- Sensitivity of query (kind of function that we are adding noise to)
- Desired Epsilon (how much privacy leakage we want to allow. We have a ceratin budget of epsilon that we want to stay underneath). 
- Desired Delta (how much privacy leakage we want to allow) <br></br>

**Laplacian mechanizm** is a function that is being augmented with Laplacian noise in this way, forming the mechanizm  **"M"** If we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).<br></br>
**TO DO** Search for different private randomized mechanisms that can be apprpriate for different use cases.

In [0]:
#import torch
import torch
import numpy as np

epsilon = 0.0001    # epsilon we are spending per query

In [0]:
def create_parallel_db(db, index):

    return torch.cat((db[0:index], 
                      db[index+1:]))

In [0]:
def create_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = create_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [0]:
def create_db_and_parallels(num_els):
    
    db = torch.rand(num_els) > 0.5
    pdbs = create_parallel_dbs(db)
    
    return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(100)

In [0]:
# If db consists of 0-2, the sensitivity would double to 2. !!!!!!!!!!!!!!!!
# because the output would change and if we remove entry, it coul dbe as high as 2
# sum(db * 2)
def sum_query(db):
  return db.sum()

In [0]:
def laplacian_mechanism(db, query, sensitivity):
  
  beta = sensitivity / epsilon
  # 0 here is mean centered at 0, beta is controlling teh amount of spread. 
  # it's NOT range [0,1]
  noise = torch.tensor(np.random.laplace(0, beta, 1))
  
  return query(db) + noise

In [0]:
def mean_query(db):
  return torch.mean(db.float())

In [70]:
mean_query(db)

tensor(0.6100)

In [72]:
# laplacian_mechanism(db, sum_query, 1) is right below and above sum_query(db)
laplacian_mechanism(db, sum_query, 1)

tensor([9872.8785], dtype=torch.float64)

In [77]:
laplacian_mechanism(db, mean_query, 1/100)    # noise is smaller here

tensor([69.2208], dtype=torch.float64)

###Observation
For epsilon = 0.5 laplacian_mechanism(db, mean_query, 1/100) was dancing around  sum_query(db) range. 

For epsilon = 0.0001 laplacian_mechanism(db, mean_query, 1/100) is all over the place. It's very random because we're not leaking hardly any information. 

**1. The lower the epsilon, we are increasing teh amount of noise that we have to add in order to be able to get really constraining level of privacy protection.**

**2. The more noise we add, the more protection, the more plausible deniability that we gave each participant in our survey, the more randomize our results we're looking.**